In [1]:
from collections import defaultdict

# Sets
CENTRAL = ["W0"]                     # supply node (infinite stock)
WAREHOUSES = ["W1", "W2"]
RETAILERS = ["R1", "R2", "R3", "R4"]
ENTITIES = CENTRAL + WAREHOUSES + RETAILERS

# 15‑period planning horizon
T = list(range(1, 16))

# Parameters
# Holding cost per unit per period
holding_cost = {"W1": 0.20, "W2": 0.20,
                "R1": 0.60, "R2": 0.60, "R3": 0.60, "R4": 0.60}

# Fixed ordering (setup) cost
ordering_cost = {e: 30 for e in WAREHOUSES + RETAILERS}

# Initial inventory
initial_inventory = {"W1": 1_200, "W2": 1_100,
                     "R1": 350, "R2": 450, "R3": 500, "R4": 600}

# Retail demands (constant each period for simplicity – edit as needed)
demand = {("R1", t): 50 for t in T}
demand.update({("R2", t): 60 for t in T})
demand.update({("R3", t): 70 for t in T})
demand.update({("R4", t): 80 for t in T})

# Unit transportation cost
tc = defaultdict(float)
tc.update({("W0", "W1"): 0.55, ("W0", "W2"): 0.22})
tc.update({("W1", "R1"): 0.22, ("W1", "R2"): 0.20,
           ("W1", "R3"): 0.32, ("W1", "R4"): 0.38})
tc.update({("W2", "R1"): 0.68, ("W2", "R2"): 0.52,
           ("W2", "R3"): 0.34, ("W2", "R4"): 0.10})

# Transport lead times (periods)
lead = defaultdict(int)
for w in WAREHOUSES:
    lead[("W0", w)] = 3
    for r in RETAILERS:
        lead[(w, r)] = 3

# Replenishment‑profile matrix 
# profiles[p][t] == 1 if profile p allows orders in period t
profiles = {}
# p1 = order every period
profiles[1] = {t: 1 for t in T}
# p2 = odd periods
profiles[2] = {t: int(t % 2 == 1) for t in T}
# p3 = even periods
profiles[3] = {t: int(t % 2 == 0) for t in T}
# p4/p5/p6 = 3‑cycle starting at 1/2/3
profiles[4] = {t: 1 if t % 3 == 1 else 0 for t in T}
profiles[5] = {t: 1 if t % 3 == 2 else 0 for t in T}
profiles[6] = {t: 1 if t % 3 == 0 else 0 for t in T}
profiles[7]  = {t: 1 if (t-1) % 4 == 0 else 0 for t in T}   # 1,5,9,13
profiles[8]  = {t: 1 if (t-2) % 4 == 0 else 0 for t in T}   # 2,6,10,14
profiles[9]  = {t: 1 if (t-3) % 4 == 0 else 0 for t in T}   # 3,7,11,15
profiles[10] = {t: 1 if  t      % 4 == 0 else 0 for t in T} # 4,8,12
profiles[11] = {t: 1 if (t-1) % 5 == 0 else 0 for t in T}   # 1,6,11
profiles[12] = {t: 1 if (t-2) % 5 == 0 else 0 for t in T}   # 2,7,12
profiles[13] = {t: 1 if (t-3) % 5 == 0 else 0 for t in T}   # 3,8,13
profiles[14] = {t: 1 if (t-4) % 5 == 0 else 0 for t in T}   # 4,9,14
profiles[15] = {t: 1 if  t      % 5 == 0 else 0 for t in T} # 5,10,15

# Big‑M (sufficiently large)
BIG_M = 10_000

capacity = {"W1":  1200, "W2":  1200,
            "R1":  450, "R2":  500,
            "R3":  550, "R4":  600}

# Emission factors (kg CO2 per unit shipped)
emission_factors = {
    ("W0","W1"): 0.50,
    ("W0","W2"): 0.30,
    ("W1","R1"): 0.20,
    ("W1","R2"): 0.18,
    ("W1","R3"): 0.25,
    ("W1","R4"): 0.28,
    ("W2","R1"): 0.35,
    ("W2","R2"): 0.30,
    ("W2","R3"): 0.22,
    ("W2","R4"): 0.15,
}

# Weight on emissions in the objective (€/kg CO2)
emission_cost_weight = 0.25

In [2]:
from data import WAREHOUSES, RETAILERS, initial_inventory

s_level = {"W1": 480, "W2": 0, "R1": 250, "R2": 270, "R3": 360, "R4": 440}
S_level = {"W1": 1200, "W2": 1100, "R1": 500, "R2": 630, "R3": 710, "R4": 840}

def load_initial_solution(m):
    # Set profiles from Table 6
    profile_choices = {"W1": 6, "W2": 1, "R1": 10, "R2": 10, "R3": 10, "R4": 10}
    for e in WAREHOUSES + RETAILERS:
        p = profile_choices[e]
        v = m.getVarByName(f"x_profile[{e},{p}]")
        if v:
            v.Start = 1

    # Period-1 orders if below s
    for e in WAREHOUSES + RETAILERS:
        inv_t1 = initial_inventory[e]  # No shipments arrive in t1
        if inv_t1 < s_level[e] and profile_choices[e] == 1:  # Profile #1 allows t=1
            need = S_level[e] - inv_t1
            y = m.getVarByName(f"y_order[{e},1]")
            if y:
                y.Start = 1
            q = m.getVarByName(f"orderQty[{e},1]")
            if q:
                q.Start = need
            if e in WAREHOUSES:
                shp = m.getVarByName(f"ship_w0_w[W0,{e},1]")
                if shp:
                    shp.Start = need
            else:
                # Split across warehouses (simplified: all from W1 or W2)
                home = "W1" if e in ("R1", "R2", "R3") else "W2"
                shp = m.getVarByName(f"ship_w_r[{home},{e},1]")
                if shp:
                    shp.Start = need

In [3]:
from gurobipy import Model, GRB, quicksum
from data import (CENTRAL, WAREHOUSES, RETAILERS, ENTITIES, T, holding_cost,
                  ordering_cost, initial_inventory, demand, tc, lead, profiles, BIG_M, capacity, 
                emission_factors, emission_cost_weight)

def build_base_model(with_capacity_caps: bool = True, with_emissions: bool = False) -> Model:
    m = Model("PeriodicReview_sS")
    m.setParam("OutputFlag", 0)

    # Decision variables
    inv = m.addVars(ENTITIES, T, name="inv", lb=0)
    ship_w0_w = m.addVars(["W0"], WAREHOUSES, T, name="ship_w0_w", lb=0)
    ship_w_r = m.addVars(WAREHOUSES, RETAILERS, T, name="ship_w_r", lb=0)
    x_profile = m.addVars(WAREHOUSES + RETAILERS, profiles.keys(), vtype=GRB.BINARY, name="x_profile")
    x_allowed = m.addVars(WAREHOUSES + RETAILERS, T, vtype=GRB.BINARY, name="x_allowed")
    x_required = m.addVars(WAREHOUSES + RETAILERS, T, vtype=GRB.BINARY, name="x_required")
    y_order = m.addVars(WAREHOUSES + RETAILERS, T, vtype=GRB.BINARY, name="y_order")
    q = m.addVars(WAREHOUSES + RETAILERS, T, name="orderQty", lb=0)

    # (s,S) levels from Table 6
    s_level = {"W1": 480, "W2": 0, "R1": 250, "R2": 270, "R3": 360, "R4": 440}
    S_level = {"W1": 1200, "W2": 1100, "R1": 500, "R2": 630, "R3": 710, "R4": 840}

    # Objective
    transport_cost = (
        quicksum(tc["W0", w] * ship_w0_w["W0", w, t] for w in WAREHOUSES for t in T) +
        quicksum(tc[w, r] * ship_w_r[w, r, t] for w in WAREHOUSES for r in RETAILERS for t in T)
    )
    obj = (
        quicksum(ordering_cost[e]*y_order[e,t] for e in WAREHOUSES+RETAILERS for t in T)
      + quicksum(holding_cost[e]*inv[e,t]     for e in holding_cost for t in T)
      + transport_cost
    )


    if with_emissions:
        # 1) Track total emissions
        total_emissions = m.addVar(name="total_emissions", lb=0)
        # 2) Link it to all shipments
        m.addConstr(
            total_emissions
            == quicksum(emission_factors[i,j] * ship_w0_w[i,j,t]
                        for i,j in emission_factors
                        if i=="W0" and j in WAREHOUSES
                        for t in T)
             + quicksum(emission_factors[w,r] * ship_w_r[w,r,t]
                        for w,r in emission_factors
                        if w in WAREHOUSES and r in RETAILERS
                        for t in T),
            name="emissions_calc"
        )
        # 3) No hard cap any more
        #    (m.addConstr(total_emissions <= emission_budget, name="emission_cap"))
        # 4) But we *do* penalize every kg CO₂ in the objective
        obj += emission_cost_weight * total_emissions
    
    m.setObjective(obj, GRB.MINIMIZE)

    # Inventory balance – warehouses
    for w in WAREHOUSES:
        for t in T:
            in_ship = ship_w0_w["W0", w, t - lead["W0", w]] if t - lead["W0", w] >= 1 else 0
            out_ship = quicksum(ship_w_r[w, r, t] for r in RETAILERS)
            if t == 1:
                m.addConstr(inv[w, t] == initial_inventory[w] + in_ship - out_ship, name=f"inv_bal_{w}_{t}")
            else:
                m.addConstr(inv[w, t] == inv[w, t - 1] + in_ship - out_ship, name=f"inv_bal_{w}_{t}")

    # Inventory balance – retailers
    for r in RETAILERS:
        for t in T:
            in_ship = quicksum(ship_w_r[w, r, t - lead[w, r]] if t - lead[w, r] >= 1 else 0 for w in WAREHOUSES)
            demand_t = demand[(r, t)]
            if t == 1:
                m.addConstr(inv[r, t] == initial_inventory[r] + in_ship - demand_t, name=f"inv_bal_{r}_{t}")
            else:
                m.addConstr(inv[r, t] == inv[r, t - 1] + in_ship - demand_t, name=f"inv_bal_{r}_{t}")

    # Initial and final inventory equality
    for e in WAREHOUSES + RETAILERS:
        m.addConstr(inv[e, T[-1]] == initial_inventory[e], name=f"final_inv_{e}")

    # Profile choice
    for e in WAREHOUSES + RETAILERS:
        m.addConstr(quicksum(x_profile[e, p] for p in profiles) == 1, name=f"oneProfile_{e}")
        for t in T:
            m.addConstr(x_allowed[e, t] == quicksum(profiles[p][t] * x_profile[e, p] for p in profiles), name=f"allowed_{e}_{t}")
            m.addConstr(y_order[e, t] <= x_allowed[e, t], name=f"y_leq_allowed_{e}_{t}")

    # (s,S) policy
    for e in WAREHOUSES + RETAILERS:
        for t in T:
            # Replenishment required if inventory < s
            m.addConstr(-BIG_M * x_required[e, t] + 0.0001 <= inv[e, t] - s_level[e], name=f"req1_{e}_{t}")
            m.addConstr(inv[e, t] - s_level[e] <= BIG_M * (1 - x_required[e, t]), name=f"req2_{e}_{t}")
            # Order if allowed and required
            m.addConstr(x_allowed[e, t] + x_required[e, t] <= y_order[e, t] + 1, name=f"order1_{e}_{t}")
            m.addConstr(y_order[e, t] <= x_allowed[e, t], name=f"order2_{e}_{t}")
            m.addConstr(y_order[e, t] <= x_required[e, t], name=f"order3_{e}_{t}")
            # Order quantity
            m.addConstr(q[e, t] <= BIG_M * y_order[e, t], name=f"q1_{e}_{t}")
            m.addConstr(q[e, t] >= S_level[e] - inv[e, t] - BIG_M * (1 - y_order[e, t]), name=f"q2_{e}_{t}")
            m.addConstr(q[e, t] <= S_level[e] - inv[e, t] + BIG_M * (1 - y_order[e, t]), name=f"q3_{e}_{t}")
            # Shipments
            if e in WAREHOUSES:
                m.addConstr(ship_w0_w["W0", e, t] == q[e, t], name=f"qFlow_{e}_{t}")
            else:
                m.addConstr(quicksum(ship_w_r[w, e, t] for w in WAREHOUSES) == q[e, t], name=f"qFlow_{e}_{t}")

    # Capacity extension
    if with_capacity_caps:
        for e, cap in capacity.items():
            for t in T:
                m.addConstr(inv[e, t] <= cap, name=f"cap_{e}_{t}")

    m.update()
    return m

In [4]:
"""
Entry-point script:
* Builds the MILP once (with capacity-caps extension already inside).
* Solves twice — cold start vs. warm start.
* Prints solve times side-by-side.
"""

import time
from rich import print
from tabulate import tabulate

from model import build_base_model
from init_heuristic import load_initial_solution
from gurobipy import GRB

# helper: solve & time
def time_solve(model, label: str) -> float:
    """
    Optimize a Gurobi model and return elapsed wall-clock time (seconds).
    """
    t0 = time.perf_counter()
    model.optimize()
    if model.Status == GRB.INFEASIBLE:
        model.computeIIS()
        model.write("model_iis.ilp")
        for c in model.getConstrs():
            if c.IISConstr:
                print("IIS constraint:", c.ConstrName)
    elapsed = time.perf_counter() - t0

    status = model.Status            # 2 == OPTIMAL
    obj = model.ObjVal if status == 2 else float("nan")
    gap = model.MIPGap if status == 2 else float("nan")

    print(f"[bold cyan]{label}[/]  status={status}  "
          f"obj={obj:.2f}  gap={gap:.4f}  time={elapsed:.2f}s")
    return elapsed

def main() -> None:
    rows = []

    # 1) Base model (no capacity caps)
    m_base_cold = build_base_model(with_capacity_caps=False)
    t_base_cold = time_solve(m_base_cold, "Base Cold-start")
    rows.append(("base", "cold", f"{t_base_cold:.2f}"))

    m_base_warm = build_base_model(with_capacity_caps=False)
    load_initial_solution(m_base_warm)
    t_base_warm = time_solve(m_base_warm, "Base Warm-start")
    rows.append(("base", "warm", f"{t_base_warm:.2f}"))

    # 2) Extended model (with capacity caps)
    m_ext_cold = build_base_model(with_capacity_caps=True, with_emissions=True)
    t_ext_cold = time_solve(m_ext_cold, "Extended Cold-start")
    rows.append(("extended", "cold", f"{t_ext_cold:.2f}"))

    m_ext_warm = build_base_model(with_capacity_caps=True, with_emissions=True)
    load_initial_solution(m_ext_warm)
    t_ext_warm = time_solve(m_ext_warm, "Extended Warm-start")
    rows.append(("extended", "warm", f"{t_ext_warm:.2f}"))

    # summary table
    print("\n[bold]Solve-time comparison (seconds)[/]")
    print(tabulate(rows, headers=["Model", "Init", "Time"], tablefmt="github"))


if __name__ == "__main__":
    main()

Set parameter Username
Set parameter LicenseID to value 2673712
Academic license - for non-commercial use only - expires 2026-05-30


Base Cold-start  status=2  obj=14811.00  gap=0.0000  time=0.11s

Base Warm-start  status=2  obj=14811.00  gap=0.0000  time=0.09s

Extended Cold-start  status=2  obj=15371.50  gap=0.0000  time=0.03s

Extended Warm-start  status=2  obj=15371.50  gap=0.0000  time=0.04s

Solve-time comparison (seconds)

| Model    | Init   |   Time |
|----------|--------|--------|
| base     | cold   |   0.11 |
| base     | warm   |   0.09 |
| extended | cold   |   0.03 |
| extended | warm   |   0.04 |